# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from joblib import parallel_backend
from IPython.display import display
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet') # download for lemmatization
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cristian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cristian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cristian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Cristian\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM DisasterResponse', con = engine)
X = df['message']
y = df[df.columns.tolist()[2:]]

y_columns = y.columns

y = y.to_numpy()
X = X.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 2, test_size = 0.2)

### 2. Write a tokenization function to process your text data

In [3]:
cachedStopWords = stopwords.words("english")

def tokenize(text):
    # Normalize text and get ride of punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Actual tokenization
    words = word_tokenize(text)
    # Getting rid of common words
    words = [w for w in words if w not in cachedStopWords]
    # 1st pass at reducing words to base form using nouns
    words_noun_lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    # 2nd pass at reducing words to base form using verbs
    words_verb_lemmed = [WordNetLemmatizer().lemmatize(w, pos = 'v') for w in words_noun_lemmed]
    return words_verb_lemmed

priming_the_tokenizer = tokenize('A short piece of text used for priming the tokenize function. This is done for preventing some errors, that occur when this is first used, from happening during \
                                hyper-parameter tuning.')

print(priming_the_tokenizer)

['short', 'piece', 'text', 'use', 'prim', 'tokenize', 'function', 'do', 'prevent', 'error', 'occur', 'first', 'use', 'happen', 'hyper', 'parameter', 'tune']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def print_and_write_report(filename: str, y_test, y_preds, columns):

    with open("model_reports/" + filename + ".txt", "w") as f:

        for index, column in enumerate(columns):
            f.write('Current column is {}.\n\n'.format(column))
            print('Current column is {}.\n'.format(column))
            report = classification_report(y_test[:,index], y_preds[:,index], zero_division = 0)
            f.write(report)
            print(report)

In [5]:
basic_random_forest = Pipeline([('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])), \
            ('clasification_model', MultiOutputClassifier(RandomForestClassifier(), n_jobs = 10))])

In [6]:
basic_MultinomialNB = Pipeline([('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])), \
            ('clasification_model', MultiOutputClassifier(MultinomialNB()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
basic_MultinomialNB.fit(X_train, y_train)
with parallel_backend('threading'):
    basic_random_forest.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_preds_basic_MultinomialNB = basic_MultinomialNB.predict(X_test)

print_and_write_report("Basic_MultinomialNB", y_test, y_preds_basic_MultinomialNB, y_columns)

Current column is related.

              precision    recall  f1-score   support

         0.0       0.50      0.00      0.01      1187
         1.0       0.77      1.00      0.87      4012
         2.0       0.00      0.00      0.00        43

    accuracy                           0.77      5242
   macro avg       0.42      0.33      0.29      5242
weighted avg       0.70      0.77      0.66      5242

Current column is request.

              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91      4359
         1.0       0.69      0.01      0.02       883

    accuracy                           0.83      5242
   macro avg       0.76      0.51      0.47      5242
weighted avg       0.81      0.83      0.76      5242

Current column is offer.

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      5215
         1.0       0.00      0.00      0.00        27

    accuracy                           0.99   

In [9]:
# with parallel_backend('threading'):
y_preds_basic_random_forest = basic_random_forest.predict(X_test)

print_and_write_report("Basic_Radom_Forest", y_test, y_preds_basic_random_forest, y_columns)

Current column is related.

              precision    recall  f1-score   support

         0.0       0.30      0.05      0.08      1187
         1.0       0.77      0.97      0.86      4012
         2.0       0.25      0.02      0.04        43

    accuracy                           0.75      5242
   macro avg       0.44      0.35      0.33      5242
weighted avg       0.66      0.75      0.67      5242

Current column is request.

              precision    recall  f1-score   support

         0.0       0.84      0.98      0.90      4359
         1.0       0.41      0.06      0.10       883

    accuracy                           0.83      5242
   macro avg       0.62      0.52      0.50      5242
weighted avg       0.77      0.83      0.77      5242

Current column is offer.

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      5215
         1.0       0.00      0.00      0.00        27

    accuracy                           0.99   

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
        'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'text_pipeline__vect__max_df': (0.5, 1.0),
        'text_pipeline__vect__max_features': (None, 5000),
        'text_pipeline__tfidf__use_idf': (True, False),
        'text_pipeline__tfidf__sublinear_tf': (True, False)
        # 'clasification_model__estimator__alpha': (0.5, 1)
    }

cv = GridSearchCV(basic_random_forest, parameters, verbose = 2)

with parallel_backend('threading'):
    cv.fit(X_train, y_train)

display(cv.best_params_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END text_pipeline__tfidf__sublinear_tf=True, text_pipeline__tfidf__use_idf=True, text_pipeline__vect__max_df=0.5, text_pipeline__vect__max_features=5000, text_pipeline__vect__ngram_range=(1, 1); total time=51.2min
[CV] END text_pipeline__tfidf__sublinear_tf=True, text_pipeline__tfidf__use_idf=True, text_pipeline__vect__max_df=0.5, text_pipeline__vect__max_features=5000, text_pipeline__vect__ngram_range=(1, 1); total time=53.9min
[CV] END text_pipeline__tfidf__sublinear_tf=True, text_pipeline__tfidf__use_idf=True, text_pipeline__vect__max_df=0.5, text_pipeline__vect__max_features=5000, text_pipeline__vect__ngram_range=(1, 1); total time=54.2min
[CV] END text_pipeline__tfidf__sublinear_tf=True, text_pipeline__tfidf__use_idf=True, text_pipeline__vect__max_df=0.5, text_pipeline__vect__max_features=5000, text_pipeline__vect__ngram_range=(1, 2); total time=54.6min
[CV] END text_pipeline__tfidf__sublinear_tf=True, text_pipelin

{'text_pipeline__tfidf__sublinear_tf': False,
 'text_pipeline__tfidf__use_idf': False,
 'text_pipeline__vect__max_df': 0.5,
 'text_pipeline__vect__max_features': None,
 'text_pipeline__vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
Improved_multinomialNB = Pipeline([('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize, max_df = 0.5, ngram_range = (1,2))),
                ('tfidf', TfidfTransformer(sublinear_tf = True, use_idf = False))
            ])), \
            ('clasification_model', MultiOutputClassifier(MultinomialNB()))])

Improved_multinomialNB.fit(X_train, y_train)
y_preds_Improved_multinomialNB = Improved_multinomialNB.predict(X_test)

print_and_write_report("Improved_MultinomialNB", y_test, y_preds_Improved_multinomialNB, y_columns)

Current column is related.

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1187
         1.0       0.77      1.00      0.87      4012
         2.0       0.00      0.00      0.00        43

    accuracy                           0.77      5242
   macro avg       0.26      0.33      0.29      5242
weighted avg       0.59      0.77      0.66      5242

Current column is request.

              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91      4359
         1.0       0.00      0.00      0.00       883

    accuracy                           0.83      5242
   macro avg       0.42      0.50      0.45      5242
weighted avg       0.69      0.83      0.76      5242

Current column is offer.

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      5215
         1.0       0.00      0.00      0.00        27

    accuracy                           0.99   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.